# Load wandb metrics and create nicer plots

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from copy import deepcopy
from pathlib import Path

import dotenv
import molfeat
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from hydra import compose, initialize
from hydra.core.global_hydra import GlobalHydra
from hydra.utils import instantiate
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import WandbLogger
from omegaconf import DictConfig, OmegaConf, open_dict

import wandb
from src import utils
from src.models.jump_cl import BasicJUMPModule
from src.utils import instantiate_evaluator_list

In [3]:
dotenv.load_dotenv()

True

In [5]:
for i in range(1, 4):
    if not Path(f"../cpjump{i}/jump/").exists():
        print(f"Mounting cpjump{i}...")
        os.system(f"sshfs bioclust:/projects/cpjump{i}/ ../cpjump{i}")
    else:
        print(f"cpjump{i} already mounted.")

cpjump1 already mounted.
cpjump2 already mounted.
cpjump3 already mounted.


In [4]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gabriel-watkinson-work (jump_models). Use `wandb login --relogin` to force relogin


True

In [5]:
api = wandb.Api()

run = api.run("jump_models/first_real_runs/q06qjpvj")

In [7]:
metrics_dataframe = run.history()
# metrics_dataframe.to_csv("metrics.csv")

In [16]:
run.config["ckpt_path"]

'/import/pr_cpjump1/jump/logs/train/runs/2023-08-26_20-35-21/checkpoints/epoch_049.ckpt'

## Load config

In [6]:
ckpt_str = "../cpjump1/jump/logs/train/multiruns/{run}/checkpoints/epoch_{epoch:0>3}.ckpt"
single_run_ckpt_str = "../cpjump1/jump/logs/train/runs/{run}/checkpoints/epoch_{epoch:0>3}.ckpt"

run_dict = {
    "small1": (run := "2023-08-16_11-59-26/0", "small_jump_cl", epoch := 43, ckpt_str.format(run=run, epoch=epoch)),
    "small": (run := "2023-08-17_13-32-50/0", "small_jump_cl", epoch := 41, ckpt_str.format(run=run, epoch=epoch)),
    "med": (run := "2023-08-07_11-55-54", "med_jump_cl", epoch := 5, ckpt_str.format(run=run, epoch=epoch)),
    "big": (run := "2023-08-01_11-37-40", "big_jump_cl", epoch := 1, ckpt_str.format(run=run, epoch=epoch)),
    "new_small": (
        run := "2023-08-22_17-15-50",
        "fp_small",
        epoch := 43,
        single_run_ckpt_str.format(run=run, epoch=epoch),
    ),
}

In [7]:
run, experiment, epoch, ckpt = run_dict["new_small"]

In [8]:
os.system(f"cat ../cpjump1/jump/logs/train/runs/{run}/.hydra/overrides.yaml");

- experiment=fp_small
- trainer=gpu
- trainer.devices=[1]
- data.num_workers=16
- callbacks=default


In [9]:
os.listdir(f"../cpjump1/jump/logs/train/runs/{run}/checkpoints")

['last.ckpt', 'epoch_043.ckpt']

In [10]:
initialize(version_base=None, config_path="../configs/")

hydra.initialize()

In [46]:
cfg = compose(
    config_name="train.yaml",
    overrides=[
        "evaluate=true",
        "paths.projects_dir=..",
        f"paths.output_dir=../cpjump1/jump/logs/train/multiruns/{run}",
        "experiment=fp_small",
        "data.batch_size=4",
        # "model/molecule_encoder=gin_masking.yaml",
        "trainer.devices=1",
        # "eval.moa_image_task.datamodule.data_root_dir=../",
    ],
)
print(OmegaConf.to_yaml(cfg))

task_name: train
tags:
- small_jump_cl
- fingerprints
- clip_like
- ${model.image_encoder.instance_model_name}
train: true
test: true
evaluate: true
compile: false
ckpt_path: null
seed: 12345
data:
  compound_transform:
    _target_: src.modules.compound_transforms.fp_transform.FPTransform
    fps:
    - maccs
    - ecfp
    compound_str_type: inchi
    params:
      ecfp:
        radius: 2
  _target_: src.models.jump_cl.datamodule.BasicJUMPDataModule
  batch_size: 4
  num_workers: 24
  pin_memory: null
  prefetch_factor: 3
  drop_last: true
  transform:
    _target_: src.modules.transforms.DefaultJUMPTransform
    _convert_: object
    size: 128
    dim:
    - -2
    - -1
  force_split: false
  splitter:
    _target_: src.splitters.ScaffoldSplitter
    train: 1024
    test: 256
    val: 128
    retrieval: 0
  use_compond_cache: false
  data_root_dir: ${paths.projects_dir}/
  split_path: ${paths.split_path}/fp_small3/
  dataloader_config:
    train:
      batch_size: ${data.batch_size}

## Instantiate datamodule

In [12]:
dm = instantiate(cfg.data)

/home/gwatk/miniconda3/envs/jump_models/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/gwatk/miniconda3/envs/jump_models/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Plea

In [105]:
dm.prepare_data()

KeyboardInterrupt: 

In [14]:
dm.setup("train")

In [34]:
dl = dm.train_dataloader()

/home/gwatk/miniconda3/envs/jump_models/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [35]:
b = next(iter(dl))

## Instantiate model

In [1]:
cfg.model["_target_"] += ".load_from_checkpoint"
with open_dict(cfg.model):
    cfg.model["checkpoint_path"] = ckpt

NameError: name 'cfg' is not defined

In [38]:
model = instantiate(cfg.model, map_location="cuda:0")

In [39]:
model

BasicJUMPModule(
  (image_encoder): CNNEncoder(
    (backbone): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (drop_block): Identity()
          (act1): ReLU(inplace=True)
          (aa): Identity()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act2): ReLU(inplace=True)
        )
        (1): BasicBlock(
      

## Instantiate evaluators

In [47]:
evaluators = instantiate_evaluator_list(cfg.eval, cross_modal_module=model, logger=None)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
No callback configs found! Skipping..
GPU available: True (cuda), used: True
TPU availab

In [50]:
evaluators

EvaluatorList(
    n_evaluators=7,
    evaluators=
        (JumpMOAImageModule) Evaluator
		(Tox21Module) Evaluator
		(LipoModule) Evaluator
		(EsolModule) Evaluator
		(BBBPModule) Evaluator
		(HIVModule) Evaluator
		(IDRRetrieval) IDRRetrievalEvaluator
)

In [53]:
evaluators[1].datamodule.compound_transform

In [36]:
logger = utils.instantiate_loggers(cfg.get("logger"))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gabriel-watkinson-work (jump_models). Use `wandb login --relogin` to force relogin


In [83]:
Trainer?

Init signature:
Trainer(
    *,
    accelerator: Union[str, lightning.pytorch.accelerators.accelerator.Accelerator] = 'auto',
    strategy: Union[str, lightning.pytorch.strategies.strategy.Strategy] = 'auto',
    devices: Union[List[int], str, int] = 'auto',
    num_nodes: int = 1,
    precision: Union[Literal[64, 32, 16], Literal['16-mixed', 'bf16-mixed', '32-true', '64-true'], Literal['64', '32', '16', 'bf16']] = '32-true',
    logger: Union[lightning.pytorch.loggers.logger.Logger, Iterable[lightning.pytorch.loggers.logger.Logger], bool, NoneType] = None,
    callbacks: Union[List[lightning.pytorch.callbacks.callback.Callback], lightning.pytorch.callbacks.callback.Callback, NoneType] = None,
    fast_dev_run: Union[int, bool] = False,
    max_epochs: Optional[int] = None,
    min_epochs: Optional[int] = None,
    max_steps: int = -1,
    min_steps: Optional[int] = None,
    max_time: Union[str, datetime.timedelta, Dict[str, int], NoneType] = None,
    limit_train_batches: Union[int, 

In [90]:
[x for x in trainer.loggers[2].experiment.__dir__() if "log" in x]

['log_code',
 '_log',
 'log',
 '_log_job_artifact_with_image',
 'log_artifact',
 '_log_artifact',
 '_assert_can_log_artifact',
 '_footer_log_dir_info']